In [2]:
import ast
import re
import pandas as pd
from datasets import load_dataset

In [3]:
cocktails_dataset = load_dataset("csv", data_files="cocktails_all_new.csv")

Using custom data configuration default-6290b2396cc66501
Reusing dataset csv (C:\Users\lecover\.cache\huggingface\datasets\csv\default-6290b2396cc66501\0.0.0\6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e)
100%|██████████| 1/1 [00:00<00:00, 47.75it/s]
Using custom data configuration erwanlc--cocktails_recipe-6ed27d67a557fd3e


Downloading: 100%|██████████| 2.50M/2.50M [00:00<00:00, 2.56MB/s]
100%|██████████| 1/1 [00:00<00:00, 510.19it/s]


Dataset csv downloaded and prepared to C:\Users\lecover\.cache\huggingface\datasets\csv\erwanlc--cocktails_recipe-6ed27d67a557fd3e\0.0.0\6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e. Subsequent calls will reuse this data.


100%|██████████| 1/1 [00:00<00:00, 66.83it/s]


In [63]:

def create_input(example):
    input_ingredients = [item[1] for item in example["ingredients"]]
    input_ingredients = ",".join(input_ingredients)
    input_ingredients = re.sub(r" \([^()]*\)", "", input_ingredients)
    return {"inputs": input_ingredients}

def create_output(example):
    ingredients = [" ".join(item) for item in example["ingredients"]]
    ingredients = "\n".join(ingredients)
    output = example["recipe"] + "\n" + ingredients
    output = re.sub(r" \([^()]*\)", "", output)
    return {"targets": output}

cocktails_dataset = cocktails_dataset.map(lambda x: {"ingredients": ast.literal_eval(x["ingredients"])})
cocktails_dataset = cocktails_dataset.map(lambda x: {"recipe": x["recipe"] if x["recipe"] else "Untitled"})
cocktails_dataset = cocktails_dataset.map(create_input)
cocktails_dataset = cocktails_dataset.map(create_output)

Loading cached processed dataset at C:\Users\lecover\.cache\huggingface\datasets\csv\default-6290b2396cc66501\0.0.0\6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e\cache-02cb1992f88c99dd.arrow
Loading cached processed dataset at C:\Users\lecover\.cache\huggingface\datasets\csv\default-6290b2396cc66501\0.0.0\6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e\cache-bf549cc5551414b0.arrow
100%|██████████| 6956/6956 [00:01<00:00, 5981.69ex/s]


In [64]:
cocktails_dataset = cocktails_dataset["train"].train_test_split(train_size=0.8, seed=42)
cocktails_dataset

DatasetDict({
    train: Dataset({
        features: ['title', 'glass', 'garnish', 'recipe', 'ingredients', 'inputs', 'targets'],
        num_rows: 5564
    })
    test: Dataset({
        features: ['title', 'glass', 'garnish', 'recipe', 'ingredients', 'inputs', 'targets'],
        num_rows: 1392
    })
})

In [65]:
cocktails_dataset = cocktails_dataset.remove_columns(["glass", "title", "garnish", "recipe", "ingredients"])

In [66]:
cocktails_dataset.save_to_disk("cocktails_datasets")

Flattening the indices: 100%|██████████| 2/2 [00:00<00:00, 62.60ba/s]
